In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import numpy as np
from scipy import stats
import param
import panel as pn
pn.extension('tabulator')
pn.extension('plotly')

import hvplot.pandas
import plotly.express as px
# hvplot.extension('plotly')


In [3]:
df = pd.read_csv('dashboard_data.csv')
df = df.drop(columns='Unnamed: 0')
df.sample(10)

,Date,Amplitude,Time,Temperature,Dew_Point,Humidity,Wind,Wind_Speed,Pressure,Condition,Region,Month,Year
16732,2019-04-28,10.000000,12:00 PM,23.888889,23.888889,100.0,VAR,1.0,29.79,Light Rain with Thunder,Sai Gon,4,2019
63520,2020-03-21,13.888889,1:00 PM,31.111111,22.777778,62.0,ESE,12.0,29.85,Fair,Da Nang,3,2020
63106,2020-03-18,17.222222,8:30 PM,26.111111,22.777778,83.0,SE,5.0,29.88,Partly Cloudy,Da Nang,3,2020
122249,2021-05-08,11.111111,8:00 PM,27.777778,25.000000,84.0,VAR,1.0,29.76,Fair,Da Nang,5,2021
92607,2020-10-11,8.888889,7:00 AM,26.111111,26.111111,100.0,VAR,3.0,29.64,Mostly Cloudy,Sai Gon,10,2020
31772,2019-08-12,11.111111,7:00 PM,32.222222,22.777778,59.0,SSE,7.0,29.48,Partly Cloudy,Ha Noi,8,2019
84273,2020-08-13,7.777778,10:30 AM,27.777778,23.888889,79.0,E,5.0,29.69,Partly Cloudy,Ha Noi,8,2020
12984,2019-04-02,17.222222,1:00 AM,27.222222,22.777778,79.0,ESE,9.0,29.85,Fair,Sai Gon,4,2019
94369,2020-10-23,15.000000,1:30 PM,26.111111,17.777778,61.0,NW,6.0,29.82,Mostly Cloudy,Da Nang,10,2020
158782,2022-01-18,17.777778,6:00 AM,15.000000,11.111111,77.0,CALM,0.0,30.17,Fair,Ha Noi,1,2022


In [8]:
df.columns

Index(['Date', 'Amplitude', 'Time', 'Temperature', 'Dew_Point', 'Humidity',
       'Wind', 'Wind_Speed', 'Pressure', 'Condition', 'Region', 'Month',
       'Year'],
      dtype='object')

## Biên độ nhiệt

In [7]:
# df_new = df.groupby("Date")['Temperature'].max
# df['Amplitude'] = df['Temperature'] - df['Temperature']

In [8]:
# biên độ nhiệt
# amp = pd.DataFrame(df.groupby("Date")['Temperature'].max() - df.groupby("Date")['Temperature'].min())
# amp.columns = ['Amplitude']
# fig = px.line(amp, x=amp.index,y='Amplitude', title='Hướng gió', width=500, height=600 )
# fig.show()

In [9]:
# dfa = pd.DataFrame(df['Wind'].value_counts())
# # dfa.columns = ["name","value"]
# fig = px.pie(dfa,values='Wind', title='Hướng gió', names=dfa.index, width=500,
#     height=600 )
# fig.show()

## DashBoard

In [6]:
# Years = list(set(df['Year']))
# Years[1]

In [7]:
# dfa = df[df['Region']== 'Da Nang']
# dfa[dfa['Year']== 2020]

In [12]:
# fig = px.bar(dfc,x='Condition', title='Hướng gió', y=dfc.index, width=500, height=600)
# fig.show()

In [7]:
# set(df['year'])
Years = list(set(df['Year']))
Regions = list(set(df['Region']))
inputs = ['Month','Temperature','Dew_Point', 'Humidity','Amplitude','Wind_Speed', 'Pressure']

class Dashboard(param.Parameterized):
    Region = param.Selector(Regions, default= Regions[1])
    select = param.Selector(Years, default= Years[1])
    X_variable = param.Selector(inputs, default=inputs[2])
    Y_variable = param.Selector(inputs, default=inputs[1])
    
    @param.depends('X_variable', 'Y_variable', 'select', 'Region')
    def scatter_plot(self):
        dfa = df[df['Region']==self.Region]
        return  dfa[dfa['Year']== self.select].hvplot.scatter(x=self.X_variable, y=self.Y_variable, width=500, height=450)
    
    @param.depends('Y_variable', 'select', 'Region')
    def line_plot(self):
        dfa = df[df['Region']==self.Region]
        return  dfa[dfa['Year']== self.select].hvplot.line('Date', self.Y_variable, width=1500)
    
    @param.depends('Y_variable', 'select', 'Region')
    def violin_plot(self):
        dfa = df[df['Region']==self.Region]
        return  dfa[dfa['Year']== self.select].hvplot.violin(self.Y_variable, 'Month', width=1500)
    
    # Subplot
    
    @param.depends('Y_variable', 'select', 'Region')
    def subline_plot_1(self):
        dfa = df[df['Region']==self.Region]
        return  dfa[dfa['Year']== self.select].groupby('Month').mean().hvplot(x='Month', y=['Temperature', 'Amplitude'],
                 value_label='Mean', subplots=True, width=750)
    
    @param.depends('Y_variable', 'select', 'Region')
    def subline_plot_2(self):
        dfa = df[df['Region']==self.Region]
        return  dfa[dfa['Year']== self.select].groupby('Month').mean().hvplot(x='Month', y=['Dew_Point', 'Humidity', 'Wind_Speed'],
                 value_label='Mean', subplots=True, width=500)
    
    @param.depends('Y_variable', 'select', 'Region')
    def box_plot(self):
        dfa = df[df['Region']==self.Region]
        df_box = dfa[dfa['Year']== self.select].groupby('Date').max()
        return df_box.hvplot.box(by='Month', y= self.Y_variable, legend=False, width=500)
    
    @param.depends('Y_variable', 'select', 'Region')
    def pie_plot(self):
        dfb = df[df['Region']==self.Region]
        dfa = pd.DataFrame(dfb[dfb['Year']== self.select]['Wind'].value_counts())
        fig = px.pie(dfa,values='Wind', names=dfa.index)
        return pn.pane.Plotly(fig, config={'responsive': True} , width=500)
    
    @param.depends('Y_variable', 'select', 'Region')
    def barh_condition(self):
        dfa = df[df['Region']==self.Region]
        dfc = pd.DataFrame(dfa[dfa['Year']== self.select].groupby('Condition').count()).sort_values('Time')[10:]
        return dfc.hvplot.barh(y='Time', width=500)
    
    @param.depends('select', 'Region')
    def amp_plot(self):
        dfa = df[df['Region']==self.Region]
        amp = pd.DataFrame(dfa[dfa['Year']== self.select].groupby("Date")['Temperature'].max() - dfa[dfa['Year']== self.select].groupby("Date")['Temperature'].min())
        amp.columns = ['Amplitude']
        return amp.hvplot.line('Date', self.Y_variable, width=1500)
        # fig = px.line(amp, x=amp.index,y='Amplitude', title='Hướng gió')
        # return pn.pane.Plotly(fig, config={'responsive': True}, width=1500)
    
    @param.depends('select', 'Region')
    def heat_map(self):
        dfa = df[df['Region'] == self.Region]
        z =dfa[dfa['Year']== self.select][['Temperature', 'Dew_Point', 'Humidity', 'Pressure']].corr()
        fig = px.imshow(z, text_auto=True,  color_continuous_scale='Blues')
        return pn.pane.Plotly(fig, config={'responsive': True} , width=500)
    
    @param.depends('select', 'Region')
    def win_speed(self):
        dfa = df[df['Region']==self.Region]
        wind = dfa[dfa['Year']== self.select].groupby('Wind_Speed').count()
        return wind.hvplot.bar(y='Time', width=500)
            
        
         
        
    
    def panel(self):
        return pn.Row(
            pn.Param(self, width=200, name = "Plot Settings", sizing_mode="fixed"),
            pn.Column(
                self.subline_plot_1,
                self.subline_plot_2,
                pn.Row(self.heat_map,self.pie_plot, self.scatter_plot),
                pn.Row(self.win_speed, self.box_plot,self.barh_condition),
            # self.amp_plot,
            # self.subline_plot,
            # self.box_plot,
            # self.pie_plot,
            # self.scatter_plot,
            # self.violin_plot
            # self.line_plot,
            # self.barh_condition,
            # self.heat_map,
            # self.win_speed,self.barh_condition
            )
        )
dashboard = Dashboard(name='Weather_Dashboard')
component = dashboard.panel()
component

Row
    [0] Param(Dashboard, name='Plot Settings', sizing_mode='fixed', width=200)
    [1] Column
        [0] ParamMethod(method)
        [1] ParamMethod(method)
        [2] Row
            [0] ParamMethod(method)
            [1] ParamMethod(method)
            [2] ParamMethod(method)
        [3] Row
            [0] ParamMethod(method)
            [1] ParamMethod(method)
            [2] ParamMethod(method)

In [69]:
pn.template.FastListTemplate(site="Panel", title="Weather_Dashboard", 
                             main=[
                                 component 
                             ]).servable();

In [ ]:
# df['Month'] = pd.DatetimeIndex(df['Date']).month.astype(str)
# df['year'] = pd.DatetimeIndex(df['Date']).year.astype(str)
# df_box = df.groupby('Date').max()
# sns.boxplot(data = df_box, x ="Month", y="Dew_Point")
# plt.title('Daily Maximum Dew Point Observations')